<a href="https://colab.research.google.com/github/sidhunt/CUDA-sample/blob/main/CUDA_assgn_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 4 submission by Siddhant Badola (ID-1154586).
The following code was executed in Google Colab, kindly run the code in the same to get similar results.

In [ ]:
!nvidia-smi
import torch
torch.cuda.is_available()
import tensorflow as tf
tf.test.gpu_device_name()
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

Sat Nov 21 01:32:04 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

9 point coding question where An*Bn=Cn

In [ ]:
%%cu
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include<time.h>
#include <stdio.h>
#include <sys/time.h>
using namespace std;
void MM(float *,float *,float *,int,int,int,int,int,int);
                                      //Kindly input dimensions below in hA,wA,hB,wB,N as described in the assignment to get the desired results and CPU/GPU run-time
 const int hA=500; //row of A
 const   int wA=500;//col of A
 const int hB=500;//row of B
 const   int wB=400;//col of B
 const   int N=110;  // N
 struct timeval t1,t2;
float* A=new float[hA*wA];
float* B=new float[hB*wB];
  float* C=new float[hA*wB];
  float* result=new float[hA*wB];
  double total=0.0;
void initAB(int m2, int n2){
    int i2=0;
        for(i2=0 ;i2<(m2*n2);i2++){      
        A[i2]=(rand()%10);
         B[i2]=(rand()%10);    
    }
}
void initResC(int m1 ,int k1){
    int c1=0;
    for(c1=0;c1<(m1*k1);c1++){
        result[c1]=0.0;
        C[c1]=0.0;
    }
}

__global__ void MMGpuKernel(float *Ad,float *Bd,float *Cd,const int wA1,const int wB1){
    float Cval=0.0;
    int r=blockIdx.y*blockDim.y+threadIdx.y;
    int c=blockIdx.x*blockDim.x+threadIdx.x;
    for(int z=0;z<wA1;z++){
        float Adele=Ad[r*wA1+z];
        float Bdele=Bd[z*wB1+c];
        Cval +=(float)(Adele*Bdele);
    }
    Cd[r*wB1+c]=Cval;
}
void MMGpu(float *A,float *B,float *C,int num,int count,const int m,const int n,const int j,const int k){
        gettimeofday(&t1,0);
    float *Ad=new float[m*n];
    float *Bd=new float[j*k];
    float *Cd=new float[m*k];
    cudaMalloc(&Ad,sizeof(int)*m*n);
    cudaMemcpy(Ad,A,sizeof(int)*m*n,cudaMemcpyHostToDevice);
    cudaMalloc(&Bd,sizeof(int)*j*k);
    cudaMemcpy(Bd,B,sizeof(int)*j*k,cudaMemcpyHostToDevice);
    cudaMalloc(&Cd,sizeof(int)*m*k);
    cudaMemcpy(Cd,C,sizeof(int)*m*k,cudaMemcpyHostToDevice);
   dim3 dimBlock(2,1,1);
   dim3 dimGrid;
    dimGrid.x = ((k + dimBlock.x - 1)/dimBlock.x);
    dimGrid.y = ((m + dimBlock.y - 1)/dimBlock.y);
    
    MMGpuKernel<<<dimGrid,dimBlock>>>(Ad,Bd,Cd,n,k);
    gettimeofday(&t2,0);
    cudaMemcpy(C,Cd,sizeof(int)*m*k,cudaMemcpyDeviceToHost);
     double executed=(1000000.0*(t2.tv_sec-t1.tv_sec)+t2.tv_usec-t1.tv_usec)/1000.0;
    printf("GPU Execution time for N=%d: %3.4f ms (C%d=%0.1f)\n",count+1,executed,count+1,C[count]);
    total+=executed;
    cudaFree(Ad);
    cudaFree(Bd);
    cudaFree(Cd);
}
void execCPU(float* result,float* A,float* B,int hA2,int wA2,int wB2,int count2){
        gettimeofday(&t1,0);
        int i1,j1,k1;
        double a1,b1;
 for( i1=0;i1<hA2;i1++){
     for(j1=0;j1< wB2;j1++){
         int sum=0;
         for( k1=0;k1<wA2;k1++){
             a1 =A[i1*wA2+k1];
             b1=B[k1*wB2+j1];
             sum+= a1*b1;
         }
         result[i1*wB2+j1]=sum;
     }
 }
       gettimeofday(&t2,0);

   double executed1=(1000000.0*(t2.tv_sec-t1.tv_sec)+t2.tv_usec-t1.tv_usec)/1000.0;
   printf("CPU Execution time for N=%d: %3.4f ms (C%d=%0.1f)\n",count2+1,executed1,count2+1,result[count2]);
}
int main(void){
    int x1=0;
    int a1=0;
    int count1=0;
                        srand(time(NULL));
    for(a1=0;a1<N;a1++,count1++){
            initAB(hA,wA);
            initResC(hA,wB);
        execCPU(result,A,B,hA,wA,wB,count1);
       MMGpu(A,B,C,N,count1,hA,wA,hB,wB);
        
    }
    printf("Average GPU time = %f ms",total/N);
    return 0;
}

CPU Execution time for N=1: 657.8440 ms (C1=9680.0)
GPU Execution time for N=1: 93.0830 ms (C1=9680.0)
CPU Execution time for N=2: 678.3350 ms (C2=10301.0)
GPU Execution time for N=2: 1.0320 ms (C2=10301.0)
CPU Execution time for N=3: 653.4660 ms (C3=9686.0)
GPU Execution time for N=3: 1.0920 ms (C3=9686.0)
CPU Execution time for N=4: 650.6590 ms (C4=10495.0)
GPU Execution time for N=4: 1.0640 ms (C4=10495.0)
CPU Execution time for N=5: 648.6790 ms (C5=9864.0)
GPU Execution time for N=5: 1.1020 ms (C5=9864.0)
CPU Execution time for N=6: 657.6770 ms (C6=10288.0)
GPU Execution time for N=6: 1.0920 ms (C6=10288.0)
CPU Execution time for N=7: 652.2510 ms (C7=9416.0)
GPU Execution time for N=7: 1.1500 ms (C7=9416.0)
CPU Execution time for N=8: 654.8390 ms (C8=10729.0)
GPU Execution time for N=8: 1.1080 ms (C8=10729.0)
CPU Execution time for N=9: 654.8590 ms (C9=10110.0)
GPU Execution time for N=9: 1.1780 ms (C9=10110.0)
CPU Execution time for N=10: 647.9820 ms (C10=9829.0)
GPU Execution tim

(1 point) comparing python and cuda code of 3rd Assignment part II.

In [ ]:
%%cu
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include<time.h>
#include <stdio.h>
#include <sys/time.h>
using namespace std;
void MM(float *,float *,float *,int,int,int,int,int,int);
                                      //Kindly input dimensions below in hA,wA,hB,wB,N as described in the assignment to get the desired results and CPU/GPU run-time
 const int hA=500; //row of A
 const   int wA=500;//col of A
 const int hB=500;//row of B
 const   int wB=400;//col of B
 const   int N=110;  // N
 struct timeval t1,t2;
float* A=new float[hA*wA];
float* B=new float[hB*wB];
  float* C=new float[hA*wB];
  float* result=new float[hA*wB];
  double ttlTime=0.0;
void initA(int m2, int n2){
    int i2=0;
        for(i2=0 ;i2<(m2*n2);i2++){      
        A[i2]=(rand()%10);    
    }
}
void initResC(int m1 ,int k1){
    int c1=0;
    for(c1=0;c1<(m1*k1);c1++){
        result[c1]=0.0;
        C[c1]=0.0;
    }
}

__global__ void MMGpuKernel(float *Ad,float *Bd,float *Cd,const int wA1,const int wB1){
    float Cval=0.0;
    int r=blockIdx.y*blockDim.y+threadIdx.y;
    int c=blockIdx.x*blockDim.x+threadIdx.x;
    for(int z=0;z<wA1;z++){
        float Adele=Ad[r*wA1+z];
        float Bdele=Bd[z*wB1+c];
        Cval +=(float)(Adele*Bdele);
    }
    Cd[r*wB1+c]=Cval;
}
void MMGpu(float *A,float *B,float *C,int num,int count,const int m,const int n,const int j,const int k){
        gettimeofday(&t1,0);
    float *Ad=new float[m*n];
    float *Bd=new float[j*k];
    float *Cd=new float[m*k];
    cudaMalloc(&Ad,sizeof(int)*m*n);
    cudaMemcpy(Ad,A,sizeof(int)*m*n,cudaMemcpyHostToDevice);
    cudaMalloc(&Bd,sizeof(int)*j*k);
    cudaMemcpy(Bd,B,sizeof(int)*j*k,cudaMemcpyHostToDevice);
    cudaMalloc(&Cd,sizeof(int)*m*k);
    cudaMemcpy(Cd,C,sizeof(int)*m*k,cudaMemcpyHostToDevice);
   dim3 dimBlock(2,1,1);
   dim3 dimGrid;
    dimGrid.x = ((k + dimBlock.x - 1)/dimBlock.x);
    dimGrid.y = ((m + dimBlock.y - 1)/dimBlock.y);
    
    MMGpuKernel<<<dimGrid,dimBlock>>>(Ad,Bd,Cd,n,k);
    gettimeofday(&t2,0);
    cudaMemcpy(C,Cd,sizeof(int)*m*k,cudaMemcpyDeviceToHost);
     double executed=(1000000.0*(t2.tv_sec-t1.tv_sec)+t2.tv_usec-t1.tv_usec)/1000.0;
    printf("GPU Execution time for N=%d: %3.4f ms (C%d=%0.1f)\n",count+1,executed,count+1,C[count]);
    ttlTime+=executed;
    cudaFree(Ad);
    cudaFree(Bd);
    cudaFree(Cd);
}
void execCPU(float* result,float* A,float* B,int hA2,int wA2,int wB2,int count2){
        gettimeofday(&t1,0);
        int i1,j1,k1;
        double a1,b1;
 for( i1=0;i1<hA2;i1++){
     for(j1=0;j1< wB2;j1++){
         int sum=0;
         for( k1=0;k1<wA2;k1++){
             a1 =A[i1*wA2+k1];
             b1=B[k1*wB2+j1];
             sum+= a1*b1;
         }
         result[i1*wB2+j1]=sum;
     }
 }
       gettimeofday(&t2,0);

   double executed1=(1000000.0*(t2.tv_sec-t1.tv_sec)+t2.tv_usec-t1.tv_usec)/1000.0;
   printf("CPU Execution time for N=%d: %3.4f ms (C%d=%0.1f)\n",count2+1,executed1,count2+1,result[count2]);
}
int main(void){
    int x1=0;
    int a1=0;
    int count1=0;
                        srand(time(NULL));
    for( x1=0;x1<(hB*wB);x1++){
        B[x1]=(rand()%10);
        
    }
    for(a1=0;a1<N;a1++,count1++){
            initA(hA,wA);
            initResC(hA,wB);
        execCPU(result,A,B,hA,wA,wB,count1);
       MMGpu(A,B,C,N,count1,hA,wA,hB,wB);
        
    }
    printf("Average GPU time = %f ms",ttlTime/N);
    return 0;
}

CPU Execution time for N=1: 693.0470 ms (C1=10450.0)
GPU Execution time for N=1: 95.8760 ms (C1=10450.0)
CPU Execution time for N=2: 692.7740 ms (C2=9794.0)
GPU Execution time for N=2: 1.1320 ms (C2=9794.0)
CPU Execution time for N=3: 686.0140 ms (C3=10308.0)
GPU Execution time for N=3: 1.1990 ms (C3=10308.0)
CPU Execution time for N=4: 691.2150 ms (C4=9772.0)
GPU Execution time for N=4: 1.1460 ms (C4=9772.0)
CPU Execution time for N=5: 686.8000 ms (C5=10372.0)
GPU Execution time for N=5: 1.0960 ms (C5=10372.0)
CPU Execution time for N=6: 678.4880 ms (C6=9799.0)
GPU Execution time for N=6: 1.0930 ms (C6=9799.0)
CPU Execution time for N=7: 686.4160 ms (C7=9576.0)
GPU Execution time for N=7: 1.1480 ms (C7=9576.0)
CPU Execution time for N=8: 686.5590 ms (C8=9804.0)
GPU Execution time for N=8: 1.1250 ms (C8=9804.0)
CPU Execution time for N=9: 687.2580 ms (C9=10241.0)
GPU Execution time for N=9: 1.1290 ms (C9=10241.0)
CPU Execution time for N=10: 694.6800 ms (C10=9987.0)
GPU Execution time 

In [ ]:
import numpy as np
import datetime as dt

In [ ]:
B=np.random.rand(500,400)
avg=0.0
N=110
for i in range(1,N+1):
  A=np.random.rand(500,500)
  d1=dt.datetime.now()
  C=np.dot(A,B)
  d2=dt.datetime.now()
  d3=d2-d1
  avg=avg+(d3.microseconds/1000.0)
  print("A{0}*B takes {1}ms".format(i,d3.microseconds/1000.0))
print(f"Average Python code time= {avg/N} ms")


A1*B takes 10.595ms
A2*B takes 6.409ms
A3*B takes 5.824ms
A4*B takes 5.897ms
A5*B takes 5.642ms
A6*B takes 6.114ms
A7*B takes 5.757ms
A8*B takes 5.894ms
A9*B takes 6.07ms
A10*B takes 5.845ms
A11*B takes 5.805ms
A12*B takes 6.039ms
A13*B takes 5.767ms
A14*B takes 6.04ms
A15*B takes 8.624ms
A16*B takes 5.973ms
A17*B takes 5.975ms
A18*B takes 5.703ms
A19*B takes 6.419ms
A20*B takes 6.006ms
A21*B takes 5.904ms
A22*B takes 8.921ms
A23*B takes 5.713ms
A24*B takes 5.955ms
A25*B takes 6.118ms
A26*B takes 5.951ms
A27*B takes 5.78ms
A28*B takes 5.682ms
A29*B takes 5.774ms
A30*B takes 5.709ms
A31*B takes 5.64ms
A32*B takes 5.774ms
A33*B takes 9.882ms
A34*B takes 11.03ms
A35*B takes 9.287ms
A36*B takes 6.171ms
A37*B takes 6.086ms
A38*B takes 6.068ms
A39*B takes 5.926ms
A40*B takes 5.737ms
A41*B takes 5.762ms
A42*B takes 7.953ms
A43*B takes 5.89ms
A44*B takes 5.896ms
A45*B takes 5.807ms
A46*B takes 6.074ms
A47*B takes 5.788ms
A48*B takes 5.861ms
A49*B takes 6.014ms
A50*B takes 5.874ms
A51*B takes 6

(5 points) comparison of python and CUDA code of assignment 4.

In [ ]:
avg=0.0
N=110
for i in range(1,N+1):
  d1=dt.datetime.now()
  A=np.random.rand(500,500)
  B=np.random.rand(500,400)
  C=np.dot(A,B)
  d2=dt.datetime.now()
  d3=d2-d1
  avg=avg+(d3.microseconds/1000.0)
  print("A{0}*B{0} takes {1}ms".format(i,d3.microseconds/1000.0))
print(f"Average Python code time= {avg/N} ms")


A1*B1 takes 16.29ms
A2*B2 takes 12.295ms
A3*B3 takes 12.106ms
A4*B4 takes 11.752ms
A5*B5 takes 11.536ms
A6*B6 takes 11.85ms
A7*B7 takes 11.991ms
A8*B8 takes 11.679ms
A9*B9 takes 11.873ms
A10*B10 takes 11.955ms
A11*B11 takes 11.894ms
A12*B12 takes 11.256ms
A13*B13 takes 11.538ms
A14*B14 takes 11.4ms
A15*B15 takes 11.583ms
A16*B16 takes 11.628ms
A17*B17 takes 11.909ms
A18*B18 takes 15.603ms
A19*B19 takes 11.64ms
A20*B20 takes 11.104ms
A21*B21 takes 11.301ms
A22*B22 takes 11.125ms
A23*B23 takes 11.544ms
A24*B24 takes 11.66ms
A25*B25 takes 11.378ms
A26*B26 takes 11.675ms
A27*B27 takes 11.103ms
A28*B28 takes 11.282ms
A29*B29 takes 11.232ms
A30*B30 takes 10.892ms
A31*B31 takes 12.207ms
A32*B32 takes 12.112ms
A33*B33 takes 14.399ms
A34*B34 takes 11.612ms
A35*B35 takes 11.603ms
A36*B36 takes 13.434ms
A37*B37 takes 11.692ms
A38*B38 takes 11.454ms
A39*B39 takes 11.117ms
A40*B40 takes 11.285ms
A41*B41 takes 11.252ms
A42*B42 takes 10.798ms
A43*B43 takes 11.158ms
A44*B44 takes 11.3ms
A45*B45 takes 

We can clearly see CUDA outperforms Python code in both the cases:-

*   For **An*B=Cn** we get averge python run-time of 6.3 milliseconds while CUDA runtime of 2 milliseconds approximately.
*   For **An*Bn=Cn** we get averge python run-time of 11.8 milliseconds while CUDA runtime of 2 milliseconds approximately.

